In [56]:
import numpy as np

<p>Functions <strong>legendre</strong> and <strong>smsieve</strong> are basic functions for computing legendre symbol returning 0 if it is not defined +1 if n is quadratic residue mod p, -1 if not and a simple sieve prime test for small numbers respectively.</p>

In [57]:
def legendre(n, p): # legendre symbol (n/p)
    if n % p == 0:
        return 0
    if n > p:
        n = n % p
    if n ** (int((p - 1) / 2)) % p == 1:
        return 1
    else:
        return -1


def smsieve(n):  # prime check with classic sieving
    if n in [1, 2]:
        return 1
    for i in range(2, int(np.floor(np.sqrt(n))) + 1):
        if n % i == 0:
            return 0
    return 1

In [58]:
print(legendre(251, 3))
print(smsieve(89), smsieve(244))

-1
1 0


<p>Function <strong>Bfactors</strong> takes integer n and set B of primes along with -1 and returns an array of the powers of primes in B or 0 if n cannot be factored using these primes. Function <strong>Bset</strong> builds the set B of first k primes p that have legendre symbol (n/p) for integer n, creating the sieve core.</p>

In [59]:
def Bfactors(n, B):  # for number n and set of primes B with -1 returns the factorisation and powers of n if n is
                     # B-smooth and 0 if n is not B-smooth
    if n < 0:
        basepowers = [1]
        n = int(n * -1)
    else:
        basepowers = [0]
    B = B[1:]
    for prime in B:
        prime_power = 0
        while n % prime == 0:
            n = int(n / prime)
            prime_power += 1
        basepowers.append(prime_power)
    if n == 1:
        return basepowers
    else:
        return 0


def Bset(k, n):  # construct the set of the first k primes p that have (n/p) = +1 together with -1
    B = [-1]
    p = 2
    while k:
        while legendre(n, p) != 1 or smsieve(p) != 1:  # if the conditions are not met, got to next prime
            p += 1
        B.append(p)
        p += 1
        k = k - 1
    return B

In [60]:
print(Bfactors(74529, [-1, 2, 3, 7, 13, 89]))
print(Bfactors(23, [-1, 23]))
print(Bfactors(122, [-1, 2, 13]))

[0, 0, 2, 2, 2, 0]
[0, 1]
0


<p>The function <strong>testisiven</strong> takes a list of lists representing powers of B-smooth numbers and outputs 1 if they are linearly dependent mod 2 (the column-wise sum is even for each column) and 0 otherwise</p>

In [61]:
def testifeven(l, B):  #  l is list of lists, B is set of primes with -1
    for i in range(len(B)):
        sumcol = 0
        for j in range(len(l)):
            sumcol += l[j][i]
        if sumcol % 2 != 0:
            return 0
    return 1

<p>Calculate the number obtained by halving the added prime powers column wise from the matrix modulo n(value v)</p>

In [62]:
def halfpower(l, B, n):  #  l is list of lists, B is set of primes with -1
    halvedpowers = 1
    for i in range(len(B)):
        sumcol = 0
        for j in range(len(l)):
            sumcol += l[j][i]
        halvedpowers *= B[i] ** int(sumcol/2)
    return halvedpowers % n

In [63]:
print(halfpower([[0, 0, 2, 0, 1, 0, 0], [0, 0, 6, 0, 1, 0, 0]], [-1, 2, 3, 13, 17, 19, 29], 87463))

1377


<p>The function <strong>searchgcds</strong> for an amount of linearly dependent sets mod 2 within the rows of given matrix using binary counting i.e. count to 2^n and represent each number in binary then check if the corresponding combination. For example number 7 corresponds to 1011 meaning we check the 0th, 2nd and 3rd row. Then build u as the product of x + t's for each corresponding row in a solution, and v as the halved powers taken from adding those rows. Finally we find u - v as a candidate to do a gcd test with n and hence hope find a factor of n.</p>

In [64]:
def searchgcds(n, factorpowerlist, xplust, B, bound):  # B only used for its length in testisiven function
    nofrows = len(factorpowerlist)
    gcds = []
    for d in range(2 ** nofrows):
        if bound == 0:
            return gcds
        c = f"{d:0{nofrows}b}"
        settocheck = []  #  builds a set of rows
        choice = []  #  chooses which rows are included and which are not
        for i in range(len(c)):
            if c[i] == '1':
                choice.append(i)
        u = 1
        if len(choice) >= 2:  #  condition on how many rows we need
            for r in choice:
                settocheck.append(factorpowerlist[r])
                u *= xplust[r]
            if testifeven(settocheck, B) == 1:
                v = halfpower(settocheck, B, n)
                u %= n
                if int(np.gcd(u - v, n)) != n and int(np.gcd(u - v, n)) != 1:
                    gcds.append(int(np.gcd(u - v, n)))
                    bound -= 1
    return gcds

<p>Setting up the factorisation: The integer n, bound of factors bound and the range of x are set by the user. The final output is a factor of n.</p>

In [65]:
n = 17480581  # number to be factorised, 17480581 = 1949 * 8969
bound = 100  # bound of factors to search/combinations to try
k = int(np.ceil(np.log10(n)) + 1)  # number of primes we will use in the B set
t = int(np.floor(np.sqrt(n)))  # number t the integer part of the square root of n
B = Bset(k, n)
X = []
yx = []
xplust = []
FactorPowers = []  # list of lists with the powers of factors
for x in range(-500000, 500000):  # choose a (small) range to look for values of x then get values of y(x) for which y(x) is B-smooth
    if Bfactors((x + t) ** 2 - n, B):
        X.append(x)
        xplust.append(x + t)
        yx.append((x + t) ** 2 - n)
        FactorPowers.append(Bfactors((x + t) ** 2 - n, B))
xyx = np.stack([X, xplust, yx], axis = -1)
factorpowersmatrix = np.stack(FactorPowers)
print(f"Matrix with columns x, x + t where t is floor(sqrt(n)), and y(x) = (x + t)^2 - n:\n\n{xyx}\n")
print(f"Smooth set of primes B:\n\n{B}\n")
print(f"Matrix of powers of each prime in B:\n\n{factorpowersmatrix}\n")
print(f"At most {bound} factors of {n}:\n\n{searchgcds(n, FactorPowers, xplust, B, bound)}")

Matrix with columns x, x + t where t is floor(sqrt(n)), and y(x) = (x + t)^2 - n:

[[     -390671      -386491 149357812500]
 [     -390177      -385997 148976203428]
 [     -338796      -334616 111950386875]
 [     -218759      -214579  46026666660]
 [     -218241      -214061  45804631140]
 [     -213921      -209741  43973806500]
 [     -162989      -158809  25202817900]
 [      -97311       -93131   8655902580]
 [      -85013       -80833   6516493308]
 [      -84714       -80534   6468244575]
 [      -62954       -58774   3436902495]
 [      -57689       -53509   2845732500]
 [      -56195       -52015   2688079644]
 [      -50357       -46177   2114834748]
 [      -50058       -45878   2087310303]
 [      -49939       -45759   2076405500]
 [      -46879       -42699   1805724020]
 [      -40059       -35879   1269822060]
 [      -35176       -30996    943271435]
 [      -34989       -30809    931713900]
 [      -33585       -29405    847173444]
 [      -28296       -24116    5641